# Example tracking workflow

## Comments
- This implementation of Munkres has a flag for assignments that can't be made (DISALLOWED).  Some implementations don't have this, so you can make the assignment unlikely by setting it to inf, but after assignment, you'll still need to check this assignment hasn't been made.

## Ideas
- Cover both Munkres (dense cost matrix) and JV LAPMOD (sparse cost matrix) for linking, comparing speed and memory allocation for different number of points.  This may be one for the talk, rather than having others do it.  That way it doesn't matter what memory is available.
- Start by getting a basic (adjacent frames only, no track splitting/merging) workflow working, then start add in components


## Installing packages

In [ ]:
!pip install --user lap
!pip install --user matplotlib
!pip install --user munkres
!pip install --user pandas
!pip install --user Pillow
!pip install --user scikit-image

## Importing libraries

In [ ]:
import math
import sys
import time
import util

import numpy as np
import scipy as sp

from munkres import Munkres, print_matrix, DISALLOWED

## Getting coordinates present in frame

In [ ]:
def get_coordinates(coords, frame):
    # Identifying rows of "coords" with current frame number
    rows = coords.index[coords['T'] == frame]

    return rows

## Initialise first timepoint
All cells detected in the first timepoint can be considered "new" tracks.  As such, we give them each a unique track ID number.

In [ ]:
def initialise_first_timepoint(coords):
    # Adding a blank extra column for the track ID
    coords['TRACK_ID'] = 0

    # Getting the row indices for the first frame
    idx_first = coords['T'] == 0

    coords.loc[idx_first,'TRACK_ID'] = range(1,sum(idx_first)+1)

## Calculate linking costs

In [ ]:
def calculate_dense_cost_matrix(coords,prev_rows,curr_rows,thresh):
    # Creating the empty array
    costs = []
    
    # Iterating over each pair, calculating the cost
    for prev_row in prev_rows:
        new_costs = []
        allowed = False
        for curr_row in curr_rows:
            # Calculating cost
            curr_pt = coords.loc[curr_row]
            cost = calculate_cost(coords.loc[prev_row],coords.loc[curr_row],thresh)
            new_costs.append(cost)
            
            if cost != DISALLOWED:
                allowed = True
                
            # If there was at least one allowed value add the cost to the cost matrix
        if allowed:
            costs.append((prev_row,new_costs))
           
    return [new_costs for _,new_costs in costs], [prev_rows for prev_rows,_ in costs]

    
def calculate_cost(prev_pt,curr_pt,thresh):
    # Spatial linking (distance between two points)
    dx = (curr_pt.X-prev_pt.X)
    dy = (curr_pt.Y-prev_pt.Y)
    d = math.sqrt(dx*dx + dy*dy)
    
    # If the two points are separated by more than the linking threshold, set them to infinity ('inf')
    if d > thresh:
        d = DISALLOWED
    
    return d

# Inheriting track IDs from previous frame
If the linked point in the previous frame already has an ID assigned to it, pass this on to the linked point (this SHOULD always be the case).  If our point wasn't assigned a link, set its track ID to the smallest unused value.

In [ ]:
def assign_IDs(assignments, coords, prev_rows, curr_rows):
    for assignment in assignments:
        ID = coords.at[prev_rows[assignment[0]],'TRACK_ID']
        coords.at[curr_rows[assignment[1]],'TRACK_ID'] = ID


# Assigning track IDs to unlinked points

In [ ]:
def assign_new_IDs(coords, curr_rows):
    # Getting the maximum track ID present in coords
    max_ID = coords.TRACK_ID.max()
    
    # Iterating over all current points and assigning the next available ID if they're still 0
    for curr_row in curr_rows:
        if coords.at[curr_row,'TRACK_ID'] == 0:
            max_ID = max_ID + 1
            coords.at[curr_row,'TRACK_ID'] = max_ID

## Main workflow

In [ ]:
%%html
<style>
.output_wrapper button.btn.btn-default,
.output_wrapper .ui-dialog-titlebar {
  display: none;
}
</style>

In [ ]:
%matplotlib notebook

# Setting parameters
np.set_printoptions(precision=3,threshold=sys.maxsize)
linking_thresh = 5

# # Loading image stack
path = "..\\data\\ExampleTimeseries.tif"
images = util.load_images(path);

# Loading coordinates
path = "../data/ObjectCoordinatesNoHeader.csv"
coords = util.load_coordinates(path);

# # Getting the number of frames
n_frames = images.shape[2]
print("Loaded %i frames" % n_frames)

# Set new track IDs for each object in the first frame
initialise_first_timepoint(coords)

# Starting at frame 2, looping over each frame, linking pairs
print("")
for frame in range (1,n_frames):    
    sys.stdout.write("\rProcessing frame %i" % frame)
    
    # Get row labels for the previous and current frame
    prev_rows = get_coordinates(coords,frame-1)
    curr_rows = get_coordinates(coords,frame)
    
    # Calculate costs for each possible link, then use Munkres to determine assignments
    dense_costs, nz_prev_rows = calculate_dense_cost_matrix(coords,prev_rows,curr_rows,linking_thresh)    
    M = Munkres()
    assignments = M.compute(dense_costs)
    
    # Assigning links
    assign_IDs(assignments,coords,nz_prev_rows,curr_rows)
    assign_new_IDs(coords,curr_rows)


In [ ]:
# Adding track renders
util.show_overlay(images,coords)